마디는 1부터 시작
박 1/4 , 1/16, 위치은 0부터 시작


In [1]:
import os, csv, argparse
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
from tqdm import tqdm
from time import time

#import torchaudio
#from torchaudio import transforms
import librosa
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

#from whisper.audio import pad_or_trim
from io import StringIO
from sylk_parser import SylkParser

In [2]:
txt_path = './data/note/rhythmholic/'
ogg_path = './data/ogg/'
bpm_file_path = './data/음원 리소스 현황.xlsx'
note = 'rhythmholic'


In [3]:
class Data_UT():
    
    def root_tolist(dir_path):
        # 유니코드라 마지막에 슬러시 붙이는걸 추천 ex) dir_path = 'C:/Users/sbkim/T3/code/voice/whisper/data/ogg/'
        
        f_path = []
        
        for (root, directories, files) in os.walk(dir_path):
            for d in directories:
                d_path = os.path.join(root, d)

            for file in files:
                file_path = os.path.join(root, file)
                f_path.append(file_path)
                
        return f_path

    def ogg_load(audio_path): #ogg 파일 로드
        data, sr = librosa.load(audio_path, sr=22050)
        
        return data, sr

    def slk_to_df(slk_path, col_num): #slk 파일 로드

        slk = SylkParser(slk_path,  encoding="ANSI")
    
        fbuf = StringIO()
        slk.to_csv(fbuf)
    
        test_results = fbuf.getvalue() #slk 파일의 value 추출
    
        df = test_results.replace('"','')
        df = df.replace('\n ','').replace("\n",',').replace("enum(n,s,f)", "enum").replace("s,","s,0,") #slk 줄바꿈 될 때 \n으로 작성된다.
    
        df = df.split(",")
    
        df_ = []
        df_ = pd.DataFrame(df_, columns = df[:col_num])

        for i in range(1, len(df) // int(col_num)):
            df_.loc[i - int(1)] = df[col_num* i - 1 + 1 : col_num * (i+1)]
    
        return df_
    
    def txt_to_df(txt_path): #txt 파일 로드
    
        with open(txt_path, "r", encoding='UTF-8-sig') as file:
            strings = file.readlines()
        
        df = []
        df = pd.DataFrame(df, columns = list(strings[0].replace('\n','').split('\t')))
    
        n = 0

        for i in strings[2:]:
            i = i.replace('\n','')
            i = i.split('\t')
            df.loc[n] = i
    
            n += 1
    
        df = df.replace("", "0")
    
        return df
    
    def per_note_list(bpm_file, note):
        bpm_df = pd.read_excel(bpm_file)

        bpm_df1 = bpm_df[['ogg', str(note)]]
        bpm_df1 = bpm_df1.loc[(bpm_df.ogg !='NONE') & (bpm_df.rhythmholic !='NONE')  ] #ogg and rhyth

        bpm_df1.drop([0], axis = 0, inplace=True)

        bpm_df1 = bpm_df1.reset_index(drop=True)
        bpm_list = bpm_df1['ogg'].values.tolist()

        for i in range(len(bpm_list)):
            if 'Bpm' in bpm_list[i]:
                ogg = bpm_list[i].split('Bpm-')[1]
                bpm = ogg.split('.ogg')[0]
                if type(bpm) == float:
                    bpm_df1.loc[i,'BPM'] = float(bpm)
                else:
                    bpm_df1.loc[i,'BPM'] = float(bpm.split(' ')[0])


        for i in range(len(bpm_df1)):
            bpm_df1.loc[i,'ogg'] = bpm_df1.loc[i, str(note)].split('.')[0] + '.ogg'

        return bpm_df1
    
    def remove_audio(audio, sr): #음악 앞 부분에 키를 안치는 ogg 구간 삭제
        
        tempo = librosa.beat.tempo(onset_envelope=audio, sr=sr)
        bar = 60 / tempo * 4
        remove_sample = bar * 6 * sr
        
        total_sample = data[int(remove_sample) + 1 :]
        
        return total_sample
    
    def hol_preprocessing(df): #txt파일 라벨링

        df = df[df.동작 != 's']
        df = df.loc[:,['위치', 'Key']].sort_index()
        df.drop([0], axis = 0, inplace=True)
        df = df.reset_index(drop=True)
    
        df_1 = df #새로 생성할 프레임
    
        for i in range(1, len(df)):
            #li = list(filter(lambda x:x%2 == 0, range(int(df.loc[i-1, '위치']) + 2 , int(df.loc[i, '위치']))))
            li = list(filter(lambda x : x , range(int(df.loc[i-1, '위치']) + 1 , int(df.loc[i, '위치']))))
            
            #동작이 s인거 빼고 위치가 다 짝수인줄 알았지만 아니였다! 홀수도 있기때문에 기존에 2로 나눴지만, 1당 key가 찍히는걸로 수정
            
            for j in li:
                fi_df = pd.DataFrame([[j, 0]], columns = df.columns)
                df_1 = pd.concat([df_1, fi_df], ignore_index = True)
    
        df_1 = df_1.astype('int')
        df_1 = df_1.sort_values('위치')
        df_1 = df_1.reset_index(drop=True)
        
        #df_1.loc[df_1["Key"] == 4, "Key"] = 1
        #df_1.loc[df_1["Key"] == 6, "Key"] = 2 token으로 대체
    
        return df_1
    
    def label_tolist(df): #라벨 to list & 시작 부분부터 마디 시작까지 zero padding
        
        zero_list = list(int(df.loc[0,"위치"]) * '0')
        li = list(df.loc[:, "Key"])

        total_li = zero_list + li
        #li = df.loc[:, "Key"]
        
        return total_li
    
    def Mel_S(y, sr, frame_length = 0.025, frame_stride = 0.010): #mel_spectrogram
        # mel-spectrogram

        input_nfft = int(round(sr*frame_length))
        input_stride = int(round(sr*frame_stride))

        S = librosa.feature.melspectrogram(y=y, n_mels=64, n_fft=input_nfft, hop_length=input_stride)

        return S
    
    def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2): #spec aug
        _, n_mels, n_steps = spec.shape
        mask_value = spec.mean()
        aug_spec = spec

        freq_mask_param = max_mask_pct * n_mels
        for _ in range(n_freq_masks):
            aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

        time_mask_param = max_mask_pct * n_steps
        for _ in range(n_time_masks):
            aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

        return aug_spec
    

    def df_method(txt_df):
        for i in range(len(txt_df)):
                txt_df.loc[i, note] = str(txt_path) + txt_df.loc[i, note]
                #print(txt_df.loc[i, note])
        txt_list = list(txt_df.loc[:,note])

        return txt_list
    
    def path_method(txt_path):
        txt_files = Data_UT.root_tolist(txt_path)

        #print(ogg_files)
        txt_list = []
            

        for k in txt_files:
            if k[-4:] == '.slk':
                txt_list.append(k)
        return txt_list


    def extract_label(txt_list,  note = note):
        

        total_label = []
        total_num = []
        error_list = []
        total_df = pd.DataFrame()

        for i in range(len(txt_list)):

            df = Data_UT.slk_to_df(txt_list[i],6)
            #print(df)
            txt_df1 = Data_UT.hol_preprocessing(df)

            txt_df1 = Data_UT.label_tolist(txt_df1)
            result = ''.join(map(str, txt_df1))
            num = len(result)


            total_label.append(result)
            total_num.append(int(num))

        total_df = pd.DataFrame({str(note) : txt_list ,'label': total_label, 'len' : total_num})
        

        return total_df

    def label_padding(total_df): #label 뒷 부분 padding
        
        max_num = max(total_df.loc[:,'len'])

        for n in range(len(total_df)):
            if total_df.loc[n, 'len'] < max_num:

                sub_num = max_num - total_df.loc[n, 'len']
                sub_zero = '0' * int(sub_num)
                total_label = str(total_df.loc[n, 'label']) + sub_zero
                total_df.loc[n, 'label'] = total_label

        
        return total_df 
    
    def audio_sample_padding(audio, max_num):
        audio_ = librosa.util.fix_length(audio, size = max_num)
        return audio_
    

    def check_audio_len(df, note = note):
        length = 0
        ogg_path_list = list(df.loc[:,str(note)])
        except_ogg = []

        for i in ogg_path_list:
            i_ = i.replace('note/rhythmholic','ogg').replace('_hol','').replace('.slk','.ogg')
            audio, sr = Data_UT.ogg_load(i_)

            S = librosa.feature.melspectrogram(y=audio, sr = sr)

            aud_length = np.shape(S)[1]

            if aud_length > length:
                length = aud_length


            '''
            #일반 길이에서 짜르는거 잘못한거!!!
            audio_sample = len(audio)
            
            if audio_sample <= 5292000: #22050 * 240 (sample_rate * 초)
                if audio_sample > length:
                    length = len(audio)
            else:
                except_ogg.append(i_)


        
        li = [] #except_ogg_list

        for i in except_ogg:
            i = i.replace('.ogg','_hol.slk').replace('ogg','note/rhythmholic')
            li.append(i)
            '''        

        return length

    def except_ogg_from_df(df, except_ogg, note = note):

        df = df.loc[~(df[note].isin(except_ogg))].reset_index(drop=True)

        return df

    def get_ogg_path(df):
    
        ogg_path = []
        for i, row in df.iterrows():
            li = row.rhythmholic
            li = li.replace('note/rhythmholic','ogg').replace('_hol','').replace('.slk','.ogg')
            ogg_path.append(li)

        return ogg_path    

    def total_audio(path, max_num):

        audio, sr = Data_UT.ogg_load(path)

        S = librosa.feature.melspectrogram(y=audio, sr = sr)
        S_pad = Data_UT.audio_sample_padding(S, max_num)
            
        return S_pad
        #return S

    def concat_df(audio_sample, df):
        label = df[['label']]
        audio_df = pd.DataFrame({'audio' : audio_sample})

        total_data = pd.concat([audio_df, label], axis= 1)
        return total_data


In [4]:
class data_load():
    def total_data_load(bpm_file, note):
        df = Data_UT.per_note_list(bpm_file, note)[:10] #개수 제한
        df_list = Data_UT.df_method(df)
        label_df = Data_UT.extract_label(df_list, note)
        #label_df1 = Data_UT.label_padding(label_df)
        max_len = Data_UT.check_audio_len(label_df)
        #label_df1 = Data_UT.except_ogg_from_df(label_df, except_ogg)
        label_df1 = Data_UT.label_padding(label_df)

        return label_df1, max_len

In [5]:
df, max_len = data_load.total_data_load(bpm_file_path, note)

In [6]:
max_len

10533

In [7]:
df

,rhythmholic,label,len
0,./data/note/rhythmholic/k5723_hol.slk,0000000000000000000000000000000000000000000000...,1327
1,./data/note/rhythmholic/k5724_hol.slk,0000000000000000000000000000000000000000000000...,1085
2,./data/note/rhythmholic/k5725_hol.slk,0000000000000000000000000000000000000000000000...,2157
3,./data/note/rhythmholic/k5726_hol.slk,0000000000000000000000000000000000000000000000...,1261
4,./data/note/rhythmholic/k5728_hol.slk,0000000000000000000000000000000000000000000000...,1197
5,./data/note/rhythmholic/k5730_hol.slk,0000000000000000000000000000000000000000000000...,1597
6,./data/note/rhythmholic/k5732_hol.slk,0000000000000000000000000000000000000000000000...,1213
7,./data/note/rhythmholic/k5733_hol.slk,0000000000000000000000000000000000000000000000...,1647
8,./data/note/rhythmholic/k5735_hol.slk,0000000000000000000000000000000000000000000000...,1325
9,./data/note/rhythmholic/k5736_hol.slk,0000000000000000000000000000000000000000000000...,1069


In [8]:
for i in range(len(df)):
    df.loc[i, 'rhythmholic'] = df.loc[i, 'rhythmholic'].replace('note/rhythmholic','ogg').replace('_hol','').replace('.slk','.ogg')

In [9]:
df

,rhythmholic,label,len
0,./data/ogg/k5723.ogg,0000000000000000000000000000000000000000000000...,1327
1,./data/ogg/k5724.ogg,0000000000000000000000000000000000000000000000...,1085
2,./data/ogg/k5725.ogg,0000000000000000000000000000000000000000000000...,2157
3,./data/ogg/k5726.ogg,0000000000000000000000000000000000000000000000...,1261
4,./data/ogg/k5728.ogg,0000000000000000000000000000000000000000000000...,1197
5,./data/ogg/k5730.ogg,0000000000000000000000000000000000000000000000...,1597
6,./data/ogg/k5732.ogg,0000000000000000000000000000000000000000000000...,1213
7,./data/ogg/k5733.ogg,0000000000000000000000000000000000000000000000...,1647
8,./data/ogg/k5735.ogg,0000000000000000000000000000000000000000000000...,1325
9,./data/ogg/k5736.ogg,0000000000000000000000000000000000000000000000...,1069


In [10]:
from sklearn import preprocessing

In [11]:
class hole_Data(Dataset):
    
    def __init__(self, df, max_len, ogg_list):

        self.df = df
        self.max_len = max_len
        self.ogg_list = ogg_list
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):

        audio = Data_UT.total_audio(self.ogg_list[idx], self.max_len)
        #print(audio.size())

        label = self.df.loc[idx,'label']
        label = torch.tensor(list(map(int, label)))
        #label = torch.stack(label, dim=0)
        #return audio
        #return {'audio': audio, 'label' : label}
        return (audio, label)

In [12]:
ogg_list = Data_UT.get_ogg_path(df)

In [13]:
ogg_list

['./data/ogg/k5723.ogg',
 './data/ogg/k5724.ogg',
 './data/ogg/k5725.ogg',
 './data/ogg/k5726.ogg',
 './data/ogg/k5728.ogg',
 './data/ogg/k5730.ogg',
 './data/ogg/k5732.ogg',
 './data/ogg/k5733.ogg',
 './data/ogg/k5735.ogg',
 './data/ogg/k5736.ogg']

In [14]:
train_dl = hole_Data(df, max_len, ogg_list)

train_dl_ = DataLoader(train_dl, batch_size=2, shuffle= True, num_workers = 0)

In [15]:
np.shape(train_dl[0][1])

torch.Size([2157])

In [16]:
list(train_dl[0][1]).count(0)

1829

In [15]:
max_len

10533

In [16]:
np.shape(train_dl[0][1])

torch.Size([2157])

In [25]:
def extract_all_chars(batch):
    all_text = " ".join(batch["label"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [26]:
vocabs = extract_all_chars(df)

In [27]:
vocabs

{'vocab': [[' ', '0', '6', '4']],
 'all_text': ['...']}

In [28]:
vocab_list = list(set(vocabs["vocab"][0]) | set(vocabs["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'0': 0, '6': 1, ' ': 2, '4': 3}

In [29]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

6

In [30]:
vocab_dict

{'0': 0, '6': 1, '4': 3, '|': 2, '[UNK]': 4, '[PAD]': 5}

In [31]:
class GreedyCTCDecoder(torch.nn.Module):
    def __init__(self, labels, blank=0):
        super().__init__()
        self.labels = labels
        self.blank = blank

    def forward(self, emission: torch.Tensor) -> str:
        """Given a sequence emission over labels, get the best path string
        Args:
          emission (Tensor): Logit tensors. Shape `[num_seq, num_label]`.

        Returns:
          str: The resulting transcript
        """
        indices = torch.argmax(emission, dim=-1)  # [num_seq,]
        indices = torch.unique_consecutive(indices, dim=-1)
        indices = [i for i in indices if i != self.blank]
        return "".join([self.labels[i] for i in indices])

In [33]:
decoder = GreedyCTCDecoder(labels=vocab_dict).to(device)

In [44]:
np.shape(train_dl[0][1])

torch.Size([2157])

In [15]:
from conformer.model import Conformer

In [16]:
cuda = torch.cuda.is_available()  
device = torch.device('cuda' if cuda else 'cpu')

criterion = nn.CTCLoss().to(device)

In [17]:
#batch_size, sequence_length, dim = 2, max_len, 128
batch_size, dim = 4, 80
input_length = torch.LongTensor([2157, 3, 1])
target_length = torch.LongTensor([2157])

In [18]:
model = Conformer(num_classes=2157, 
                  input_dim=dim, 
                  encoder_dim=32, 
                  num_encoder_layers=3).to(device)

In [19]:
def training(model, train_dl, input_lengths, target_lengths, num_epochs):
    pred = []
    #criterion = nn.CTCLoss()
    criterion = nn.CrossEntropyLoss()
    #criterion = nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.00005)
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.00005)


    for epoch in tqdm(range(num_epochs), mininterval=0.01):
        running_loss = 0.0
        correct_prediction = 0
        total_prediction = 0

        for i, (data,label) in enumerate(train_dl):
            labels = label.to(device)
            inputs = data.to(device)
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            optimizer.zero_grad()
            outputs, output_lengths = model(inputs, input_lengths)
            #print(outputs)
            #trans = decoder(outputs)
            loss = criterion(outputs, labels)
            #loss = F.softmax(outputs, dim =1)
            #print(loss)
            loss.backward()
            optimizer.step()
            #scheduler.step()
            running_loss += loss.item()
            _, prediction = torch.max(outputs,1)
            pred.append(outputs)
            #print("prediction")
            #print(prediction)
            #print("label")
            #print(label)
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[1]
            #print(correct_prediction)
    
        #optimizer.swap_swa_sgd()
        num_batches = len(train_dl)
        #print(num_batches)
        #avg_loss = criterion(outputs.transpose(0, 1), targets = labels, output_lengths = output_lengths, target_lengths = target_lengths)
        avg_loss = running_loss / num_batches


        acc = correct_prediction/total_prediction
        print(f'Epoch: {epoch}, Loss: {avg_loss:.4f}, Accuracy: {acc:.4f}')

    print('Finished Training')
    return pred

        #time.sleep(0.1)
    
num_epochs=5
a = training(model, train_dl_, input_length, target_length, num_epochs)


  0%|          | 0/5 [00:02<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (62x21056 and 608x32)

In [23]:
a[0]

tensor([[[-7.7976, -7.8206, -7.6567,  ..., -7.6571, -7.6575, -7.7462],
         [-7.7895, -7.7897, -7.5845,  ..., -7.6604, -7.6774, -7.7719],
         [-7.8018, -7.8132, -7.5837,  ..., -7.6781, -7.5985, -7.7428],
         ...,
         [-7.7914, -7.7903, -7.5948,  ..., -7.7349, -7.6338, -7.6779],
         [-7.8136, -7.8293, -7.6232,  ..., -7.7132, -7.6454, -7.6650],
         [-7.7944, -7.7587, -7.6250,  ..., -7.7268, -7.6518, -7.6922]],

        [[-7.7499, -7.7836, -7.6080,  ..., -7.7353, -7.5779, -7.6917],
         [-7.8164, -7.8398, -7.6145,  ..., -7.7062, -7.6056, -7.6529],
         [-7.8001, -7.7921, -7.5763,  ..., -7.7036, -7.6084, -7.7302],
         ...,
         [-7.7452, -7.7275, -7.6023,  ..., -7.7856, -7.6421, -7.6724],
         [-7.7728, -7.7601, -7.6078,  ..., -7.7558, -7.6338, -7.6875],
         [-7.7353, -7.7251, -7.5992,  ..., -7.7919, -7.6139, -7.6510]]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward0>)

In [24]:
np.shape(a[0])

torch.Size([2, 31, 2157])

In [101]:
np.shape(a[2])

torch.Size([2, 31, 2157])

In [100]:
np.shape(a[2][0])

torch.Size([31, 2157])

In [91]:
max(a[0][0][0]

tensor(-7.1100, device='cuda:0', grad_fn=<UnbindBackward0>)

In [93]:
a[0][0][0].find(-7.1100)

AttributeError: 'list' object has no attribute 'find'

In [68]:
np.save('./data/predi.txt',a1)

In [99]:
a

[tensor([[[-7.6824, -7.7235, -7.5992,  ..., -7.8737, -7.7742, -7.6740],
          [-7.6339, -7.9091, -8.0708,  ..., -7.9338, -7.5792, -7.4553],
          [-7.8265, -7.8726, -7.8509,  ..., -7.8776, -7.6236, -7.6614],
          ...,
          [-7.6481, -7.7852, -7.8148,  ..., -7.8645, -7.5989, -7.5104],
          [-7.6313, -7.7689, -7.8622,  ..., -7.8628, -7.6599, -7.5950],
          [-7.6154, -7.7847, -7.9249,  ..., -7.8232, -7.7926, -7.5564]],
 
         [[-7.6278, -7.6087, -7.5856,  ..., -7.7199, -7.8265, -7.7112],
          [-7.7372, -7.7250, -8.1150,  ..., -8.0014, -7.3843, -7.6199],
          [-7.8522, -7.8523, -7.8662,  ..., -7.9437, -7.7490, -7.7384],
          ...,
          [-7.6415, -7.7561, -7.8512,  ..., -7.9436, -7.6349, -7.6600],
          [-7.6841, -7.7417, -7.8274,  ..., -7.8449, -7.6858, -7.5914],
          [-7.5489, -7.8715, -7.9717,  ..., -7.9556, -7.6160, -7.5791]]],
        device='cuda:0', grad_fn=<LogSoftmaxBackward0>),
 tensor([[[-7.5231, -7.6962, -7.6010,  ..., 

In [105]:
b

[tensor([[ 0,  0,  0,  ...,  0, 24,  1],
         [ 0,  0,  0,  ...,  0,  1, 12]], device='cuda:0'),
 tensor([[ 0,  0,  0,  ...,  0,  0, 30],
         [ 0,  0,  0,  ...,  0,  6, 27]], device='cuda:0'),
 tensor([[ 0,  0,  0,  ...,  0,  1,  1],
         [ 0,  0,  0,  ...,  0, 20,  9]], device='cuda:0'),
 tensor([[ 0,  0,  0,  ...,  0,  9, 26],
         [ 0,  0,  0,  ...,  0, 30, 21]], device='cuda:0'),
 tensor([[ 0,  0,  0,  ...,  0, 30, 25],
         [ 0,  0,  0,  ...,  0,  1, 15]], device='cuda:0'),
 tensor([[ 0,  0,  0,  ...,  0, 12, 23],
         [ 0,  0,  0,  ...,  0, 11, 27]], device='cuda:0'),
 tensor([[ 0,  0,  0,  ...,  0,  2, 14],
         [ 0,  0,  0,  ...,  0, 30,  1]], device='cuda:0'),
 tensor([[27,  0,  0,  ...,  0, 30, 10],
         [ 0,  0,  0,  ...,  0, 18,  1]], device='cuda:0'),
 tensor([[ 0,  0,  0,  ...,  0,  0, 10],
         [ 0,  0,  0,  ...,  0,  0, 26]], device='cuda:0'),
 tensor([[ 0,  0,  0,  ...,  0, 23, 30],
         [ 0,  0,  0,  ...,  0,  0,  2]], device='

In [107]:
b[0]

tensor([[ 0,  0,  0,  ...,  0, 24,  1],
        [ 0,  0,  0,  ...,  0,  1, 12]], device='cuda:0')

In [108]:
max(b[0][0])

tensor(30, device='cuda:0')